In [1]:
import copy
import csv
import os
import re
import warnings
import random as python_random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


2025-05-18 11:48:12.314976: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747561692.355076    7416 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747561692.365905    7416 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 11:48:12.401310: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
admissions = pd.read_csv("hosp/admissions.csv")
patients = pd.read_csv("hosp/patients.csv")
diagnoses = pd.read_csv("hosp/diagnoses_icd.csv")
icustays = pd.read_csv("icu/icustays.csv")
#chartevents = pd.read_csv("icu/chartevents.csv", chunksize=250000)  # Adjusted chunk size
labevents = pd.read_csv("hosp/labevents.csv")
discharge = pd.read_csv("discharge.csv")


In [3]:
#discharge = pd.read_csv("discharge.csv")
LVEF = discharge[discharge['text'].str.contains(r'LVEF')]

In [4]:
#LVEF_per = LVEF_new['text'].str.findall(r'LVEF.+?(?=%)')
#print(LVEF_per)
#LVEF_per = LVEF_new.loc[LVEF_new['text'].str.contains(r'LVEF.+?%', na=False)]
#print(LVEF_per)
LVEF_per = LVEF.copy()
LVEF_per['text'] = LVEF_per['text'].str.extract(r'(LVEF.+?%)')
print(LVEF_per)


               note_id  subject_id   hadm_id note_type  note_seq  \
9       10000764-DS-11    10000764  27897940        DS        11   
10      10000826-DS-17    10000826  20032235        DS        17   
18      10000980-DS-20    10000980  29654838        DS        20   
19      10000980-DS-21    10000980  26913865        DS        21   
24      10000980-DS-26    10000980  20897796        DS        26   
...                ...         ...       ...       ...       ...   
331782  19999784-DS-22    19999784  29889147        DS        22   
331783  19999784-DS-23    19999784  29956342        DS        23   
331784  19999784-DS-24    19999784  24755486        DS        24   
331790  19999840-DS-20    19999840  26071774        DS        20   
331792   19999987-DS-2    19999987  23865745        DS         2   

                  charttime            storetime         text  
9       2132-10-19 00:00:00  2132-10-19 18:50:00          NaN  
10      2146-12-12 00:00:00  2146-12-12 18:22:00    LVE

In [5]:
LVEF_gt = LVEF_per.copy()
LVEF_gt['text'] = LVEF_gt['text'].str.extract(r'(>.*[0-9]*)')
LVEF_gt = LVEF_gt.dropna(subset=['text'])
#print(LVEF_gt)
#print(LVEF_gt['text'].unique())
LVEF_gt['text'] = LVEF_gt['text'].str.extract(r'([0-9]+)')
#print(LVEF_gt['text'].unique())
gt_values = LVEF_gt.text.value_counts()
print(gt_values)

text
55    20087
60      404
75      225
65      210
70      130
50       68
40       10
45        5
47        2
43        1
20        1
64        1
30        1
32        1
80        1
71        1
25        1
58        1
Name: count, dtype: int64


In [6]:
LVEF_lt = LVEF_per.copy()
LVEF_lt['text'] = LVEF_lt['text'].str.extract(r'(<[<]*[ ]*[0-9]*)')
LVEF_lt = LVEF_lt.dropna(subset=['text'])
#print(LVEF_lt)
print(LVEF_lt['text'].unique())
LVEF_lt['text'] = LVEF_lt['text'].str.extract(r'([0-9]+)')
print(LVEF_lt['text'].unique())
lt_values = LVEF_lt.text.value_counts()
print(lt_values)

['<20' '< 20' '<40' '<35' '<30' '< 10' '<<20' '<' '< 30' '<10' '< 15'
 '<25' '< 40' '<15' '<<10' '< 25' '< 45']
['20' '40' '35' '30' '10' nan '15' '25' '45']
text
20    144
30     25
40     12
10      8
25      8
15      7
35      4
45      1
Name: count, dtype: int64


In [7]:
LVEF_et = LVEF_per.copy()
LVEF_et['text'] = LVEF_et['text'].str.extract(r'(=.*[0-9])')
LVEF_et = LVEF_et.dropna(subset=['text'])
#print(LVEF_et)
print(LVEF_et['text'].unique())
LVEF_et['text'] = LVEF_et['text'].str.extract(r'([0-9]+)')
print(LVEF_et['text'].unique())
et_values = LVEF_et.text.value_counts()
print(et_values)

['= ? 35-40' '= 25' '= 40-45' '= 64' '= 35-40' '= 65' '= 45-50' '= 59'
 '= 20' '= 68' '= 40' '= 35' '= 61' '= 79' '= 55' '= 60' '= 26' '= 15'
 '=45' '= 46' '= 40 - 45' '= 50-55' '= 72' '= 63' '= 41' '= 50' '= 21'
 '= 30' '= 45' '= 70' '=75' '= 13' '=25' '=64' '= 58' '= 36' '= 47' '= 53'
 '=40' '= 71' '= 52' '=55' '= 69' '= 57' '= 17' '= 56' '= 29' '=63' '=42'
 '= 62' '= 44' '= 55-60' '= 30 - 35' '=50' '= ? 40' '=59' '=55-60' '= 43'
 '=40-45' '= 27' '= 10 - 15' '= 67' '= <20' '= 20 - 25' '=70' '= 49'
 '= 33' '= 48' '=56' '= 15 - 20' '= 38' '=65' '=50-60' '= 45 - 50' '= 54'
 '=60' '= 60-65' '= 22' '= 25 - 30' '=30' '= 40-50' '= 37' '= 23'
 '= 15*-20' '= 42' '= >55' '= 24' '= 10' '=20' '= 66' '= 75' '= 34' '= 73'
 '= 3040' '= 14' '=58' '= 32' '=35' '= 19' '=30 to 40' '= ? 30' '= 74'
 '= 60-70' '= ?45-50' '= 77' '= ?40' '= 35- 40' '= 31' '= ?45' '= ~50'
 '= 35-45' '= < 10' '= 51' '= 50 - 55' '=45-50' '= 65-70' '=53' '= >60'
 '= 25 -30' '= 28' '= 45 to 50' '= 80' '= 78' '= 35 to 40' '= 39'


In [8]:
#print(gt_values, lt_values, et_values)
under_40 = sum([
    (LVEF_et['text'].astype(float) <= 40).sum(),
    (LVEF_gt['text'].astype(float) <= 40).sum(),
    (LVEF_lt['text'].astype(float) <= 40).sum()
])

between_40_50 = sum([
    ((LVEF_et['text'].astype(float) > 40) & (LVEF_et['text'].astype(float) <= 50)).sum(),
    ((LVEF_gt['text'].astype(float) > 40) & (LVEF_gt['text'].astype(float) <= 50)).sum(),
    ((LVEF_lt['text'].astype(float) > 40) & (LVEF_lt['text'].astype(float) <= 50)).sum()
])

over_50 = sum([
    (LVEF_et['text'].astype(float) > 50).sum(),
    (LVEF_gt['text'].astype(float) > 50).sum(),
    (LVEF_lt['text'].astype(float) > 50).sum()
])

print(f"Count under 40: {under_40}")
print(f"Count between 40 and 50: {between_40_50}")
print(f"Count over 50: {over_50}")

Count under 40: 4414
Count between 40 and 50: 2218
Count over 50: 24038


In [9]:
LVEF_all = pd.concat([pd.concat([LVEF_et, LVEF_gt], ignore_index=True),LVEF_lt], ignore_index=True)
print(LVEF_all)
LVEF_under_50 = LVEF_all[LVEF_all['text'].astype(float) <= 50]
print(LVEF_under_50)

              note_id  subject_id   hadm_id note_type  note_seq  \
0      10002495-DS-13    10002495  24982426        DS        13   
1      10004235-DS-21    10004235  24181354        DS        21   
2      10004401-DS-27    10004401  29988601        DS        27   
3      10004720-DS-11    10004720  22081550        DS        11   
4      10005817-DS-11    10005817  20626031        DS        11   
...               ...         ...       ...       ...       ...   
30683   19810932-DS-3    19810932  29764035        DS         3   
30684  19818243-DS-19    19818243  25296595        DS        19   
30685   19968075-DS-2    19968075  28592225        DS         2   
30686  19993336-DS-11    19993336  24615303        DS        11   
30687  19993336-DS-12    19993336  23077223        DS        12   

                 charttime            storetime text  
0      2141-05-29 00:00:00  2141-05-30 02:29:00   35  
1      2196-03-04 00:00:00  2196-03-06 10:57:00   25  
2      2144-02-06 00:00:00  21

In [10]:
LVEF_under_50 = LVEF_under_50.drop(['note_type', 'note_seq'], axis=1)
print(LVEF_under_50)

              note_id  subject_id   hadm_id            charttime  \
0      10002495-DS-13    10002495  24982426  2141-05-29 00:00:00   
1      10004235-DS-21    10004235  24181354  2196-03-04 00:00:00   
2      10004401-DS-27    10004401  29988601  2144-02-06 00:00:00   
4      10005817-DS-11    10005817  20626031  2132-12-20 00:00:00   
6      10010471-DS-21    10010471  21322534  2155-05-10 00:00:00   
...               ...         ...       ...                  ...   
30682  19769933-DS-16    19769933  26578430  2150-03-22 00:00:00   
30683   19810932-DS-3    19810932  29764035  2148-12-18 00:00:00   
30684  19818243-DS-19    19818243  25296595  2144-03-07 00:00:00   
30686  19993336-DS-11    19993336  24615303  2171-09-12 00:00:00   
30687  19993336-DS-12    19993336  23077223  2171-10-04 00:00:00   

                 storetime text  
0      2141-05-30 02:29:00   35  
1      2196-03-06 10:57:00   25  
2      2144-02-06 10:07:00   40  
4      2132-12-20 12:07:00   35  
6      2155-0

In [11]:
LVEF_under_50 = LVEF_under_50.rename(columns={"text": "LVEF"})
print(LVEF_under_50)


              note_id  subject_id   hadm_id            charttime  \
0      10002495-DS-13    10002495  24982426  2141-05-29 00:00:00   
1      10004235-DS-21    10004235  24181354  2196-03-04 00:00:00   
2      10004401-DS-27    10004401  29988601  2144-02-06 00:00:00   
4      10005817-DS-11    10005817  20626031  2132-12-20 00:00:00   
6      10010471-DS-21    10010471  21322534  2155-05-10 00:00:00   
...               ...         ...       ...                  ...   
30682  19769933-DS-16    19769933  26578430  2150-03-22 00:00:00   
30683   19810932-DS-3    19810932  29764035  2148-12-18 00:00:00   
30684  19818243-DS-19    19818243  25296595  2144-03-07 00:00:00   
30686  19993336-DS-11    19993336  24615303  2171-09-12 00:00:00   
30687  19993336-DS-12    19993336  23077223  2171-10-04 00:00:00   

                 storetime LVEF  
0      2141-05-30 02:29:00   35  
1      2196-03-06 10:57:00   25  
2      2144-02-06 10:07:00   40  
4      2132-12-20 12:07:00   35  
6      2155-0

In [12]:
# Filter diagnoses for relevant ICD codes
heart_failure_patients = diagnoses[diagnoses["icd_code"].isin(['I5030', '42830', 'I5031', '42831', 'I5032', '42832', 'I5033', '42833'])]

# Merge with LVEF
LVEF_patients = heart_failure_patients.merge(LVEF_under_50, on=["subject_id", "hadm_id"], how="inner")
# Merge with ICU stays
hf_icu = LVEF_patients.merge(icustays, on=["subject_id", "hadm_id"], how="inner")

# Extract unique ICU stay IDs
icu_ids = hf_icu["stay_id"].unique()
print(hf_icu, len(icu_ids))

     subject_id   hadm_id  seq_num icd_code  icd_version         note_id  \
0      10010471  29842315        5    I5033           10  10010471-DS-22   
1      10014354  29600294        8    I5032           10  10014354-DS-20   
2      10041599  22152209        3    42832            9   10041599-DS-6   
3      10127936  20688231        1    42833            9  10127936-DS-21   
4      10193065  23614190        8    I5032           10  10193065-DS-23   
..          ...       ...      ...      ...          ...             ...   
329    19794689  21650727        2    I5033           10   19794689-DS-2   
330    19796013  24367668        6    42830            9  19796013-DS-13   
331    19837737  21760644        2    42832            9  19837737-DS-10   
332    19844485  26525497        2    42831            9  19844485-DS-20   
333    19913743  27776081        2    42832            9   19913743-DS-5   

               charttime            storetime LVEF   stay_id  \
0    2155-12-07 00:00:0

In [13]:
# Define ITEMIDs for relevant vitals (e.g., Heart Rate, BP, SpO2)
# Define ITEMIDs for vitals and labs
itemid_map = {
    "Heart Rate": 220045,
    "Systolic BP": 220179,
    "Diastolic BP": 220180,
    "SpO2": 220277,
    "Temperature": 223761,
    "BMI": 226512,
    "Bicarbonate": 227443,
    "Creatinine": 220615,
    "Hemoglobin": 220228,
    "INR(PT)": 220562,
    "Platelet Count": 227457,
    "Potassium": 220640,
    "WBC Count": 220546,
    "Sodium": 220645,
    "NT-proBNP": 227444,
    "Troponin T": 227429
}

# Check unique values for 'stay_id' and 'itemid' in a sample chunk
vitals_list = []
print("Unique ICU IDs:", icu_ids)
print(f"Number of unique ICU IDs: {len(icu_ids)}")

# Define chunk size
usecols = ["stay_id", "subject_id", "hadm_id", "charttime", "itemid", "valuenum"]
chunk_size = 5000  # Reduce if memory issues persist
filtered_rows = []

# Process the file in smaller chunks
with pd.read_csv("icu/chartevents.csv", usecols=usecols, chunksize=chunk_size) as reader:
    for chunk in reader:
        chunk["charttime"] = pd.to_datetime(chunk["charttime"])  # Convert datetime in chunks
        chunk_filtered = chunk[
            chunk["stay_id"].isin(icu_ids) & chunk["itemid"].isin(itemid_map.values())
        ]
        if not chunk_filtered.empty:
            filtered_rows.append(chunk_filtered)

# Concatenate filtered chunks if any data exists
if filtered_rows:
    vitals = pd.concat(filtered_rows, ignore_index=True)
    vitals.to_csv("filtered_vitals.csv", index=False)
    print(f"Filtered data saved with {len(vitals)} rows.")
else:
    print("No matching data found.")

'''
# Load relevant data
chunk_size = 1000  # Adjust the chunk size based on your available memory
vitals_list = []
for chunk in pd.read_csv("icu/chartevents.csv", chunksize=chunk_size):
    #chunk_filtered = chunk[(chunk["stay_id"].isin(icu_ids)) & (chunk["itemid"].isin(itemid_map.values()))]
    #vitals_list.append(chunk_filtered)
    #print(vitals_list)
    print(chunk)

# Concatenate all filtered chunks
vitals = pd.concat(vitals_list, ignore_index=True)
print(vitals)  # Check if the vitals dataframe is populated
'''

Unique ICU IDs: [32119961 39864867 34813591 39916050 35158537 35988506 35149906 36650825
 33976285 36275744 37484032 31615601 39064261 36007916 37066577 32115746
 37690238 36206974 30574839 38056861 37980262 39946294 30047110 35378572
 37759830 38571466 32863045 37223807 30240703 39557479 30326504 33272807
 36839738 34903620 35406666 37339173 32357993 32970582 34755431 30721603
 31132542 37858317 36516295 37548793 39082582 38768353 35013417 33870823
 38961723 36159356 30548119 31394901 33493059 38268460 39406417 31601830
 36713654 35967314 37723752 36112143 34043983 38389114 39147811 39055079
 32216303 36030807 32106452 31653741 37074647 30792513 31305394 38083052
 36547334 31901817 34867699 36912772 39817695 36117858 31195920 35816578
 37854747 36713694 39702238 37684007 34746182 32764204 30654462 38560404
 36738588 32659288 34904311 38558137 39612812 30828325 30166913 34425328
 31558480 34645917 30056217 37128514 38845954 38425787 37201080 39026094
 38236288 33155235 37936337 3331243

/tmp/ipykernel_7416/74789829.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["charttime"] = pd.to_datetime(chunk["charttime"])  # Convert datetime in chunks
/tmp/ipykernel_7416/74789829.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["charttime"] = pd.to_datetime(chunk["charttime"])  # Convert datetime in chunks
/tmp/ipykernel_7416/74789829.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk["charttime"] = pd.to_datetime(chunk["charttime"])  # Convert datetime in chunks
/tmp/ipykernel_7416/74789829.py:35: UserWarning: Could not infer format, so 

Filtered data saved with 146090 rows.


'\n# Load relevant data\nchunk_size = 1000  # Adjust the chunk size based on your available memory\nvitals_list = []\nfor chunk in pd.read_csv("icu/chartevents.csv", chunksize=chunk_size):\n    #chunk_filtered = chunk[(chunk["stay_id"].isin(icu_ids)) & (chunk["itemid"].isin(itemid_map.values()))]\n    #vitals_list.append(chunk_filtered)\n    #print(vitals_list)\n    print(chunk)\n\n# Concatenate all filtered chunks\nvitals = pd.concat(vitals_list, ignore_index=True)\nprint(vitals)  # Check if the vitals dataframe is populated\n'

In [18]:
print(vitals)
print(patients.columns)


        subject_id   hadm_id   stay_id           charttime  itemid  valuenum
0         10010471  29842315  32119961 2155-12-04 00:00:00  220045      73.0
1         10010471  29842315  32119961 2155-12-04 00:00:00  220277      99.0
2         10010471  29842315  32119961 2155-12-04 00:02:00  220179      89.0
3         10010471  29842315  32119961 2155-12-04 00:02:00  220180      41.0
4         10010471  29842315  32119961 2155-12-04 01:00:00  220045      80.0
...            ...       ...       ...                 ...     ...       ...
146085    19913743  27776081  32445761 2131-02-03 05:31:00  220645     139.0
146086    19913743  27776081  32445761 2131-02-03 05:31:00  227443      23.0
146087    19913743  27776081  32445761 2131-02-03 05:31:00  220228      12.1
146088    19913743  27776081  32445761 2131-02-03 05:31:00  220546       7.6
146089    19913743  27776081  32445761 2131-02-03 05:31:00  227457     181.0

[146090 rows x 6 columns]
Index(['subject_id', 'gender', 'anchor_age', 'anc

In [19]:
# Add death labels
admissions["admittime"] = pd.to_datetime(admissions["admittime"])
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"])
admissions["deathtime"] = pd.to_datetime(admissions["deathtime"])

# Calculate death flags
admissions["death_within_30_days"] = (
    (admissions["deathtime"] - admissions["admittime"]).dt.days <= 30
) & admissions["deathtime"].notnull()

admissions["death_within_1_year"] = (
    (admissions["deathtime"] - admissions["admittime"]).dt.days <= 365
) & admissions["deathtime"].notnull()

# Calculate age
patients["anchor_year"] = pd.to_numeric(patients["anchor_year"], errors="coerce")
patients["anchor_age"] = pd.to_numeric(patients["anchor_age"], errors="coerce")
patients["age_admission"] = patients["anchor_age"]  # or join using admittime year if needed

# Merge patient demographics with ICU data
demo = admissions.merge(patients[["subject_id", "gender", "anchor_age"]], on="subject_id", how="left")
demo = demo[["subject_id", "hadm_id", "death_within_30_days", "death_within_1_year", "anchor_age", "gender"]]


In [20]:
# Map itemids to feature names
itemid_reverse_map = {v: k for k, v in itemid_map.items()}
vitals["label"] = vitals["itemid"].map(itemid_reverse_map)

# Sort and pivot
vitals.sort_values(by=["stay_id", "charttime"], inplace=True)
vitals_pivoted = vitals.pivot_table(
    index=["stay_id", "charttime"],
    columns="label",
    values="valuenum"
).reset_index()


In [22]:
# Resample at hourly intervals per ICU stay
vitals_pivoted['charttime'] = pd.to_datetime(vitals_pivoted['charttime'])
vitals_hourly = (
    vitals_pivoted
    .groupby("stay_id", group_keys=False)
    .apply(lambda x: x.set_index("charttime").resample("1H").mean().ffill(limit=6))
    .reset_index()
)



In [23]:
# Merge LVEF + ICU + demo
full = hf_icu.merge(demo, on=["subject_id", "hadm_id"], how="left")
full = full.merge(LVEF_under_50[["subject_id", "hadm_id", "LVEF"]], on=["subject_id", "hadm_id"], how="left")

# Merge with time-series vitals
final_dataset = vitals_hourly.merge(full, on="stay_id", how="inner")


In [32]:
# Select relevant columns
print("Columns in final_dataset:")
print(final_dataset.columns.tolist())
features = [
    "Heart Rate", "Systolic BP", "SpO2", "Temperature", "BMI",
    "Bicarbonate", "Creatinine", "Hemoglobin",
    "Platelet Count", "WBC Count", "Sodium",
    "NT-proBNP", "Troponin T"
]

scaler = MinMaxScaler()
final_dataset[features] = scaler.fit_transform(final_dataset[features])

Columns in final_dataset:
['charttime_x', 'stay_id', 'BMI', 'Bicarbonate', 'Creatinine', 'Diastolic BP', 'Heart Rate', 'Hemoglobin', 'NT-proBNP', 'Platelet Count', 'Sodium', 'SpO2', 'Systolic BP', 'Temperature', 'Troponin T', 'WBC Count', 'subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version', 'note_id', 'charttime_y', 'storetime', 'LVEF_x', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'death_within_30_days', 'death_within_1_year', 'anchor_age', 'gender', 'LVEF_y']
              charttime_x     stay_id       BMI  Bicarbonate  Creatinine  \
10    2170-03-05 00:00:00  30056217.0  0.069288          NaN         NaN   
11    2170-03-05 01:00:00  30056217.0  0.069288          NaN         NaN   
12    2170-03-05 02:00:00  30056217.0  0.069288          NaN         NaN   
13    2170-03-05 03:00:00  30056217.0  0.069288          NaN         NaN   
14    2170-03-05 04:00:00  30056217.0  0.069288     0.355556        0.25   
...                   ...         ...       ...  

In [ ]:
final_dataset.to_csv("final_heart_failure_timeseries.csv", index=False)


In [ ]:
window_size = 24  # hours
segments = []

for stay_id, group in final_dataset.groupby("stay_id"):
    if len(group) >= window_size:
        segment = group.iloc[:window_size]
        segments.append(segment)

# This produces a list of time-series windows


In [ ]:
print(segments)

In [15]:
# Ensure LVEF data is in datetime format and sorted
LVEF_all["charttime"] = pd.to_datetime(LVEF_all["charttime"])
vitals = vitals.sort_values("charttime")
LVEF_all = LVEF_all.sort_values("charttime")

# Merge vitals with gender data
vitals = vitals.merge(patients, on="subject_id", how="left")

# Merge on closest timestamp using backward fill
LVEF_vitals_merged = pd.merge_asof(
    vitals,
    LVEF_all,
    on="charttime",
    by=["subject_id", "hadm_id"],
    direction="backward"
)

# Forward-fill LVEF values for each subject
LVEF_vitals_merged["LVEF"] = LVEF_vitals_merged.groupby("subject_id")["LVEF"].ffill()

'''
# Convert charttime to datetime
vitals["charttime"] = pd.to_datetime(vitals["charttime"])
LVEF_all["charttime"] = pd.to_datetime(LVEF_all["charttime"])

# Sort by charttime (required for merge_asof)
vitals = vitals.sort_values("charttime")
LVEF_all = LVEF_all.sort_values("charttime")

print(LVEF_all)

# Merge on the closest (previous) timestamp
LVEF_vitals_merged = pd.merge_asof(
    vitals,  # Larger dataframe (vitals)
    LVEF_all,  # LVEF dataframe (smaller)
    on="charttime",
    by=["subject_id", "hadm_id"],  # Match on subject & admission ID
    direction="backward"  # Use "nearest" or "forward" if needed
)

LVEF_vitals_merged["LVEF"] = LVEF_vitals_merged.groupby("subject_id")["LVEF"].ffill()

'''

print(LVEF_vitals_merged)

         subject_id   hadm_id   stay_id           charttime  itemid  valuenum  \
0          12106780  25661012  37081503 2110-02-04 10:31:00  226512      69.3   
1          12106780  25661012  37081503 2110-02-04 12:19:00  220277     100.0   
2          12106780  25661012  37081503 2110-02-04 12:19:00  220045      77.0   
3          12106780  25661012  37081503 2110-02-04 12:25:00  223761      97.5   
4          12106780  25661012  37081503 2110-02-04 12:25:00  220045      80.0   
...             ...       ...       ...                 ...     ...       ...   
1077706    14288592  28211733  31011820 2209-03-24 19:02:00  220180      75.0   
1077707    14288592  28211733  31011820 2209-03-24 20:00:00  220277      94.0   
1077708    14288592  28211733  31011820 2209-03-24 20:00:00  220045      94.0   
1077709    14288592  28211733  31011820 2209-03-24 20:02:00  220180      79.0   
1077710    14288592  28211733  31011820 2209-03-24 20:02:00  220179     152.0   

        gender note_id stor

In [16]:
# Pivot the table to get a time series per patient
LVEF_vitals_pivot = LVEF_vitals_merged.pivot_table(
    index=["subject_id", "stay_id", "charttime"], 
    columns="itemid", 
    values="valuenum"
)

# Reset column names and index
LVEF_vitals_pivot.columns = [str(col) for col in LVEF_vitals_pivot.columns]
LVEF_vitals_pivot.reset_index(inplace=True)

# Resample to hourly data
LVEF_vitals_resampled = (
    LVEF_vitals_pivot.groupby("stay_id", group_keys=False)
    .apply(lambda group: group.set_index("charttime").resample("1H").mean().ffill())
    .reset_index()
)

# Merge gender back after pivoting
LVEF_vitals_resampled = LVEF_vitals_resampled.merge(patients, on="subject_id", how="left")

# Drop unnecessary columns if they exist
LVEF_vitals_resampled.drop(["caregiver_id", "warning", "note_id"], axis=1, inplace=True, errors="ignore")

'''
LVEF_vitals = vitals.merge(LVEF_all, on=["subject_id", "hadm_id"], how="inner")
print(LVEF_vitals)

LVEF_vitals["charttime_x"] = pd.to_datetime(LVEF_vitals["charttime_x"])
LVEF_vitals["storetime_x"] = pd.to_datetime(LVEF_vitals["storetime_x"])

# Pivot table: each ITEMID becomes a separate column
LVEF_vitals_pivot = LVEF_vitals.pivot_table(index=["subject_id", "stay_id", "charttime_x"], columns="itemid", values="valuenum")

# Reset column names
LVEF_vitals_pivot.columns = [str(col) for col in LVEF_vitals_pivot.columns]

# Reset index
LVEF_vitals_pivot.reset_index(inplace=True)

# Resample to hourly data (forward fill missing values)
LVEF_vitals_resampled = (
    LVEF_vitals_pivot.groupby("stay_id", group_keys=False)
    .apply(lambda group: group.reset_index().set_index("charttime_x").resample("1H").mean().ffill())
    .reset_index()
)

LVEF_vitals_resampled.drop(["caregiver_id", "warning", "note_id"], axis=1, inplace=True, errors="ignore")
'''
print(LVEF_vitals_resampled)

                 charttime  subject_id     stay_id  220045  220179  220180  \
0      2186-03-20 00:00:00  12168737.0  30001336.0    70.5   110.0    52.0   
1      2186-03-20 01:00:00  12168737.0  30001336.0    68.5   116.0    61.0   
2      2186-03-20 02:00:00  12168737.0  30001336.0    57.0   104.0    55.0   
3      2186-03-20 03:00:00  12168737.0  30001336.0    55.0   110.0    68.0   
4      2186-03-20 04:00:00  12168737.0  30001336.0    63.0   104.0    59.0   
...                    ...         ...         ...     ...     ...     ...   
274929 2112-12-13 15:00:00  18570637.0  39990748.0    71.0   128.0    49.0   
274930 2112-12-13 16:00:00  18570637.0  39990748.0    71.0   114.0    44.0   
274931 2112-12-13 17:00:00  18570637.0  39990748.0    73.0   134.0    67.0   
274932 2112-12-13 18:00:00  18570637.0  39990748.0    68.0   130.0    53.0   
274933 2112-12-13 19:00:00  18570637.0  39990748.0    74.0   137.0    54.0   

        220228  220277  220546  220615  220645  223761  226512 

In [ ]:
#NEW CODE

In [28]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm  # Progress bar for large operations

# Load core tables
admissions = pd.read_csv("hosp/admissions.csv")
patients = pd.read_csv("hosp/patients.csv")
diagnoses = pd.read_csv("hosp/diagnoses_icd.csv")
icustays = pd.read_csv("icu/icustays.csv")
chartevents = pd.read_csv("icu/chartevents.csv", chunksize=250000)  # Adjusted chunk size
labevents = pd.read_csv("hosp/labevents.csv")
discharge = pd.read_csv("discharge.csv")



In [43]:

print(vitals, LVEF)
# Merge admissions for age and mortality
admissions["admission_year"] = pd.to_datetime(admissions["admittime"]).dt.year
patients["dod"] = pd.to_datetime(patients["dod"], errors="coerce")
admissions["admittime"] = pd.to_datetime(admissions["admittime"])
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"])
admissions = admissions.merge(
    patients[["subject_id", "anchor_age", "anchor_year", "dod"]],
    on="subject_id",
    how="left",
    suffixes=("", "_patient")  # Avoids _x and _y issues
)
admissions["dod"] = pd.to_datetime(admissions["dod"], errors="coerce")  # Convert to datetime
admissions["dod"] = admissions["dod"].fillna(pd.Timestamp.max)  # Replace NaN with future date
admissions["death_within_30_days"] = ((admissions["dischtime"] + pd.Timedelta(days=30)) >= admissions["dod"]).astype(int)
admissions["death_within_1_year"] = ((admissions["dischtime"] + pd.Timedelta(days=365)) >= admissions["dod"]).astype(int)
admissions["age_admission"] = admissions["anchor_age"] + (admissions["admission_year"] - admissions["anchor_year"])

# Merge vitals with LVEF (closest previous measurement)
vitals.sort_values("charttime", inplace=True)
LVEF.sort_values("charttime", inplace=True)
LVEF_vitals = pd.merge_asof(vitals, LVEF, on="charttime", by=["subject_id", "hadm_id"], direction="backward")

# Forward-fill missing LVEF values per patient
LVEF_vitals["LVEF"] = LVEF_vitals.groupby("subject_id")["LVEF"].ffill()

# Merge with admissions
LVEF_vitals = LVEF_vitals.merge(admissions, on=["subject_id", "hadm_id"], how="left")

# Pivot to create time-series structure
LVEF_vitals_pivot = LVEF_vitals.pivot_table(index=["subject_id", "stay_id", "charttime"], columns="itemid", values="valuenum")
LVEF_vitals_pivot.columns = [str(col) for col in LVEF_vitals_pivot.columns]
LVEF_vitals_pivot.reset_index(inplace=True)

# Resample to hourly intervals
LVEF_vitals_pivot["charttime"] = pd.to_datetime(LVEF_vitals_pivot["charttime"])

# Resample to hourly intervals (without resetting index yet)
LVEF_vitals_resampled = (
    LVEF_vitals_pivot
    .set_index("charttime")  # Ensure charttime is the index
    .groupby("stay_id", group_keys=False)
    .resample("1H")  # Resample to 1 hour intervals
    .mean()  # Compute the mean for each hour
    .ffill()  # Forward-fill missing values
)

# Optimize memory usage
LVEF_vitals_resampled = LVEF_vitals_resampled.astype({col: "float32" for col in LVEF_vitals_resampled.columns if col not in ["subject_id", "stay_id", "charttime"]})

print(LVEF_vitals_resampled.head())


Empty DataFrame
Columns: [subject_id, hadm_id, stay_id, caregiver_id, charttime, storetime, itemid, value, valuenum, valueuom, warning]
Index: []                note_id  subject_id   hadm_id  charttime            storetime  \
66243   12024697-DS-11    12024697  20302177 2110-01-15  2110-01-19 10:31:00   
268863  18106347-DS-15    18106347  24305596 2110-01-15  2110-01-15 11:25:00   
73953   12257372-DS-21    12257372  23964251 2110-01-19  2110-01-19 11:53:00   
291493  18780420-DS-10    18780420  28977824 2110-01-22  2110-01-22 15:30:00   
158399  14779071-DS-15    14779071  20963511 2110-01-22  2110-01-22 12:54:00   
...                ...         ...       ...        ...                  ...   
124701  13774741-DS-10    13774741  27575074 2210-07-23  2210-07-24 23:12:00   
184399  15566010-DS-10    15566010  29192759 2210-09-11  2210-09-12 15:03:00   
77247    12352097-DS-8    12352097  23434751 2210-10-04  2210-10-05 15:48:00   
105230   13184296-DS-3    13184296  28785633 2211-05-0

In [29]:
# Normalize ICD codes
diagnoses["icd_code"] = diagnoses["icd_code"].str.replace(".", "", regex=True)

# Define relevant ICD codes
hf_icd_codes = {"I5030", "42830", "I5031", "42831", "I5032", "42832", "I5033", "42833"}

# Filter diagnoses for heart failure
heart_failure_patients = diagnoses[diagnoses["icd_code"].isin(hf_icd_codes)]

# Function to extract LVEF values from text
def extract_lvef(text):
    match = re.search(r'(\d+)', text)
    return int(match.group(1)) if match else np.nan

# Process LVEF values
LVEF = discharge[discharge['text'].str.contains(r'LVEF', na=False)].copy()
LVEF["LVEF"] = LVEF["text"].apply(extract_lvef)
LVEF.dropna(subset=["LVEF"], inplace=True)
LVEF = LVEF.drop(columns=["note_type", "note_seq"], errors='ignore')

# Merge LVEF with heart failure patients
LVEF_patients = heart_failure_patients.merge(LVEF, on=["subject_id", "hadm_id"], how="inner")

# Merge with ICU stays
hf_icu = LVEF_patients.merge(icustays, on=["subject_id", "hadm_id"], how="inner")

# Extract ICU stay IDs
icu_ids = set(hf_icu["stay_id"].unique())

# Define ITEMIDs for vitals and labs
itemid_map = {
    "Heart Rate": 220045,
    "Systolic BP": 220179,
    "Diastolic BP": 220180,
    "SpO2": 220277,
    "Temperature": 223761,
    "BMI": 226512,
    "Bicarbonate": 227443,
    "Creatinine": 220615,
    "Hemoglobin": 220228,
    "INR(PT)": 220562,
    "Platelet Count": 227457,
    "Potassium": 220640,
    "WBC Count": 220546,
    "Sodium": 220645,
    "NT-proBNP": 227444,
    "Troponin T": 227429
}

# Process chartevents in chunks
vitals_list = []
for chunk in tqdm(chartevents, desc="Processing chartevents"):
    chunk_filtered = chunk[(chunk["stay_id"].isin(icu_ids)) & (chunk["itemid"].isin(itemid_map.values()))]
    vitals_list.append(chunk_filtered)

# Concatenate all vitals data
vitals = pd.concat(vitals_list, ignore_index=True)

# Convert timestamps
vitals["charttime"] = pd.to_datetime(vitals["charttime"])
LVEF["charttime"] = pd.to_datetime(LVEF["charttime"])



Processing chartevents: 1732it [09:23,  3.07it/s]
